In [53]:
library(dplyr)
library(tidyr)
library(tidyverse)
library(ggplot2)
library(ggpubr)
library(ggpattern)
library(igraph)
library(ggraph)
library(purrr)
library(pheatmap)
library(circlize)
library(reshape2)
library(Seurat)
library(SeuratWrappers)
library(BSgenome.Mmusculus.UCSC.mm10)
library(Signac)
library(scCustomize)
library(reticulate)
library(Matrix)
library(viridis)
library(grid)
library(ComplexHeatmap)
library(dittoSeq)
library(patchwork)
library(ggrastr)
library(ggrepel)
library(grid)
library(cicero)

use_condaenv("~/miniconda3/envs/py3.9/bin/python3.9", required = TRUE)
setwd("/media/raghav1881/data_8tb/GitHub")
options(repr.plot.width = 12, repr.plot.height = 8)

In [52]:
hippo <- readRDS('hippocampus_multiome_2024/r_objs/integrated_multiome_chromvar.rds')
fctx <- readRDS('C9Mouse_Frontal_Cortex/r_objs/merged_multiome_motifs.rds')

In [ ]:
findLinks <- function(subset_obj){
  obj.cds <- as.cell_data_set(x = subset_obj)
  umap_coords <- subset_obj@reductions$umap.peaks
  obj.cicero <- make_cicero_cds(obj.cds, reduced_coordinates = reducedDims(obj.cds)$UMAP)
  genome <- seqlengths(BSgenome.Mmusculus.UCSC.mm10)
  genome.df <- data.frame("chr" = names(genome), "length" = genome)
  conns <- run_cicero(obj.cicero, genomic_coords = genome.df, sample_num = 100)
  CCAN_assigns <- generate_ccans(conns)
  links <- ConnectionsToLinks(conns = conns, ccans = CCAN_assigns)
  Links(subset_obj) <- links
  return(subset_obj)
}

clusterLinks <- function(obj, clusters){
  DefaultAssay(obj) <- 'peaks'
  out_list <- list()
  for (cluster in clusters){
    sub <- subset(obj, idents = cluster)
    out_list[[cluster]] <- findLinks(sub)
  }
  return(out_list)
}

ccans_fctx <- clusterLinks(fctx, c('Astro', 'L2/3 IT CTX', 'L4/5 IT CTX', 'L6 CT CTX', 'Micro', 'Oligo'))
ccans_hippo <- clusterLinks(hippo, 'Oligo')

Overlap QC metrics:
Cells per bin: 50
Maximum shared cells bin-bin: 44
Mean shared cells bin-bin: 0.532393277960209
Median shared cells bin-bin: 0



[1] "Starting Cicero"
[1] "Calculating distance_parameter value"
[1] "Running models"
[1] "Assembling connections"
[1] "Successful cicero models:  10049"
[1] "Other models: "

  Too many elements in range Zero or one element in range 
                           1                         1348 
[1] "Models with errors:  0"
[1] "Done"
[1] "Coaccessibility cutoff used: 0.14"


Overlap QC metrics:
Cells per bin: 50
Maximum shared cells bin-bin: 44
Mean shared cells bin-bin: 0.238282669648339
Median shared cells bin-bin: 0



[1] "Starting Cicero"
[1] "Calculating distance_parameter value"
[1] "Running models"
[1] "Assembling connections"
[1] "Successful cicero models:  10049"
[1] "Other models: "

  Too many elements in range Zero or one element in range 
                           1                         1348 
[1] "Models with errors:  0"
[1] "Done"
[1] "Coaccessibility cutoff used: 0.18"


Overlap QC metrics:
Cells per bin: 50
Maximum shared cells bin-bin: 44
Mean shared cells bin-bin: 0.275122275180335
Median shared cells bin-bin: 0



[1] "Starting Cicero"
[1] "Calculating distance_parameter value"
[1] "Running models"
[1] "Assembling connections"
[1] "Successful cicero models:  10049"
[1] "Other models: "

  Too many elements in range Zero or one element in range 
                           1                         1348 
[1] "Models with errors:  0"
[1] "Done"
[1] "Coaccessibility cutoff used: 0.18"


Overlap QC metrics:
Cells per bin: 50
Maximum shared cells bin-bin: 44
Mean shared cells bin-bin: 0.421910680658556
Median shared cells bin-bin: 0



[1] "Starting Cicero"
[1] "Calculating distance_parameter value"
[1] "Running models"
[1] "Assembling connections"
[1] "Successful cicero models:  10049"
[1] "Other models: "

  Too many elements in range Zero or one element in range 
                           1                         1348 
[1] "Models with errors:  0"
[1] "Done"
[1] "Coaccessibility cutoff used: 0.15"


Overlap QC metrics:
Cells per bin: 50
Maximum shared cells bin-bin: 44
Mean shared cells bin-bin: 1.05554414698689
Median shared cells bin-bin: 0



In [ ]:
dir.create('C9Mouse_Frontal_Cortex/CCAN')
dir.create('hippocampus_multiome_2024/CCAN')
saveRDS(ccans_fctx, 'C9Mouse_Frontal_Cortex/CCAN/fctx_ccans.rds')
saveRDS(ccans_hippo, 'hippocampus_multiome_2024/CCAN/hippo_ccans.rds')
dir.create('CCANs')

In [ ]:
dir.create('CCANs')

In [ ]:
# For 0.1 cutoff 
findLinks <- function(subset_obj, ident, region){
  obj.cds <- as.cell_data_set(x = subset_obj)
  umap_coords <- subset_obj@reductions$umap.peaks
  obj.cicero <- make_cicero_cds(obj.cds, reduced_coordinates = reducedDims(obj.cds)$UMAP)
  genome <- seqlengths(BSgenome.Mmusculus.UCSC.mm10)
  genome.df <- data.frame("chr" = names(genome), "length" = genome)
  conns <- run_cicero(obj.cicero, genomic_coords = genome.df, sample_num = 100)
  CCAN_assigns <- generate_ccans(conns, coaccess_cutoff_override = 0.1)
	saveRDS(CCAN_assigns, paste0('CCANs/', gsub("/", "_", ident), '_', region, '_CCANs.rds'))
  #links <- ConnectionsToLinks(conns = conns, ccans = CCAN_assigns)
  #Links(subset_obj) <- links
  return(subset_obj)
}

clusterLinks <- function(obj, clusters, region){
  DefaultAssay(obj) <- 'peaks'
  out_list <- list()
  for (cluster in clusters){
    sub <- subset(obj, idents = cluster)
    out_list[[cluster]] <- findLinks(sub, ident = cluster, region = region)
  }
  return(out_list)
}

ccans_fctx <- clusterLinks(fctx, c('L2/3 IT CTX', 'L4/5 IT CTX', 'L6 CT CTX', 'Micro', 'Oligo'), region = 'fctx')
ccans_hippo <- clusterLinks(hippo, 'Oligo', region = 'hippo')

Overlap QC metrics:
Cells per bin: 50
Maximum shared cells bin-bin: 44
Mean shared cells bin-bin: 0.238282669648339
Median shared cells bin-bin: 0



[1] "Starting Cicero"
[1] "Calculating distance_parameter value"
[1] "Running models"
[1] "Assembling connections"
[1] "Successful cicero models:  10049"
[1] "Other models: "

  Too many elements in range Zero or one element in range 
                           1                         1348 
[1] "Models with errors:  0"
[1] "Done"
[1] "Coaccessibility cutoff used: 0.1"


Overlap QC metrics:
Cells per bin: 50
Maximum shared cells bin-bin: 44
Mean shared cells bin-bin: 0.275122275180335
Median shared cells bin-bin: 0



[1] "Starting Cicero"
[1] "Calculating distance_parameter value"
[1] "Running models"
[1] "Assembling connections"
[1] "Successful cicero models:  10049"
[1] "Other models: "

  Too many elements in range Zero or one element in range 
                           1                         1348 
[1] "Models with errors:  0"
[1] "Done"
[1] "Coaccessibility cutoff used: 0.1"


Overlap QC metrics:
Cells per bin: 50
Maximum shared cells bin-bin: 44
Mean shared cells bin-bin: 0.421910680658556
Median shared cells bin-bin: 0



...

[1] "Starting Cicero"
[1] "Calculating distance_parameter value"
[1] "Running models"
[1] "Assembling connections"
[1] "Successful cicero models:  10049"
[1] "Other models: "

  Too many elements in range Zero or one element in range 
                           1                         1348 
[1] "Models with errors:  0"
[1] "Done"
[1] "Coaccessibility cutoff used: 0.1"


Overlap QC metrics:
Cells per bin: 50
Maximum shared cells bin-bin: 44
Mean shared cells bin-bin: 0.437812946756581
Median shared cells bin-bin: 0



[1] "Starting Cicero"
[1] "Calculating distance_parameter value"
[1] "Running models"
[1] "Assembling connections"
[1] "Successful cicero models:  10049"
[1] "Other models: "

  Too many elements in range Zero or one element in range 
                           1                         1348 
[1] "Models with errors:  0"
[1] "Done"
[1] "Coaccessibility cutoff used: 0.1"


Overlap QC metrics:
Cells per bin: 50
Maximum shared cells bin-bin: 44
Mean shared cells bin-bin: 0.258995214882343
Median shared cells bin-bin: 0



[1] "Starting Cicero"
[1] "Calculating distance_parameter value"
[1] "Running models"
[1] "Assembling connections"
[1] "Successful cicero models:  10022"
[1] "Other models: "

  Too many elements in range Zero or one element in range 
                           1                         1375 
[1] "Models with errors:  0"
[1] "Done"
[1] "Coaccessibility cutoff used: 0.1"


In [258]:
setwd('CCANs/')
ccans <- list.files()
ccans <- ccans[!dir.exists(ccans)]
ccans <- lapply(ccans, 
                function(x){
                  cluster <- gsub('_(fctx|hippo)_CCANs\\.rds$', '', x)
									region <- gsub('^.*_(fctx|hippo)_CCANs\\.rds$', '\\1', x)
									tmp <- readRDS(x)
                  tmp$cluster <- cluster
									tmp$region <- region
                  rownames(tmp) <- NULL
                  return(tmp)
                }) %>% bind_rows
ccans$cluster <- gsub('_','/', ccans$cluster)
ccans <- filter(ccans, region == 'fctx')

In [255]:
setwd('..')
dar_fctx <- readRDS("C9Mouse_Frontal_Cortex/DAR/DA_lr.rds")
dar_hippo <- readRDS("hippocampus_multiome_2024/DAR/DARs.RDS")[[1]]

mergeDARs <- function(dars_list, genotype) {
    clustnames <- names(dars_list$markers_C9KO)
    # Add cluster column to each dataframe
    dars_list <- lapply(seq_along(dars_list), function(i) {
        if (!is.null(dars_list[[i]])) {  # Check if the entry is not NULL
            dars_list[[i]]$cluster <- names(dars_list)[[i]]
            dars_list[[i]] <- rownames_to_column(dars_list[[i]], var = "region")
            return(dars_list[[i]])
        }
    })
    # Remove any NULL entries that might be left after the lapply
    dars_list <- dars_list[!sapply(dars_list, is.null)]
    # Merge data frames while retaining the cluster column
    merged_df <- do.call(rbind, dars_list)
    merged_df$genotype <- gsub("markers_", "", genotype)
    return(merged_df)
}

mergeAllDARs <- function(dars) {
    merged_dars <- lapply(names(dars), function(genotype) {
        if (!is.null(dars[[genotype]])) {
            mergeDARs(dars[[genotype]], genotype)
        }
    })
    # Remove any NULL entries before binding rows
    merged_dars <- merged_dars[!sapply(merged_dars, is.null)]
    merged_all_dars <- bind_rows(merged_dars)
    return(merged_all_dars)
}
fctx_df <- mergeAllDARs(dar_fctx) %>%
  filter(abs(avg_log2FC) > 0.3, p_val_adj < 0.05) %>%
	dplyr::rename(feature = region) %>%
  dplyr::mutate(region = "fctx")

hippo_df <- mergeAllDARs(dar_hippo) %>%
  filter(abs(avg_log2FC) > 0.3, p_val_adj < 0.05) %>%
	dplyr::rename(feature = region) %>%
 	dplyr::mutate(region = "hippo")

dars <- bind_rows(fctx_df, hippo_df) 
dars <- dars[dars$region == 'fctx' & dars$genotype == 'C9KO',]
dars$peak_cluster <- paste0(dars$feature, dars$cluster)

In [249]:
deg_fctx <- readRDS("C9Mouse_Frontal_Cortex/DEG/deg_master.rds")
deg_hippo <- readRDS("hippocampus_multiome_2024/DEG/deg_master.rds")

deg_fctx$region <- 'fctx'
deg_hippo$region <- 'hippo'

degs <- bind_rows(deg_fctx, deg_hippo) %>% 
	filter(cluster %in% c('Astro','Oligo','L2/3 IT CTX', 'L4/5 IT CTX', 'L6 CT CTX', 'Micro'), abs(avg_log2FC) > 0.3, p_val_adj < 0.05, region == 'fctx', genotype == 'C9KO')

In [250]:
degs

gene,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,genotype,region
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
Clu,1.651691e-67,-0.8054066,0.841,0.936,5.311838e-63,Astro,C9KO,fctx
Htra1,1.025566e-64,-0.6264868,0.722,0.850,3.298220e-60,Astro,C9KO,fctx
Apoe,1.997944e-56,-0.6649476,0.978,0.991,6.425389e-52,Astro,C9KO,fctx
Gm21860,7.675051e-48,-0.5467889,0.015,0.155,2.468296e-43,Astro,C9KO,fctx
Paqr8,1.176602e-44,-0.5660228,0.683,0.815,3.783953e-40,Astro,C9KO,fctx
Cst3,2.266493e-41,-0.6827308,0.933,0.971,7.289042e-37,Astro,C9KO,fctx
9330159F19Rik,5.398262e-38,-0.6378775,0.499,0.651,1.736081e-33,Astro,C9KO,fctx
Fam107a,2.213885e-36,-0.6094098,0.314,0.495,7.119855e-32,Astro,C9KO,fctx
Mfge8,1.089575e-30,-0.5096874,0.521,0.660,3.504073e-26,Astro,C9KO,fctx


In [259]:
ccan_peaks <- unique(ccans$Peak)
ccans$peak_cluster <- paste0(ccans$Peak, '_', ccans$cluster)
ccans$ccan_cluster <- paste0(ccans$CCAN, '_', ccans$cluster)
dars$peak_cluster <- paste0(dars$feature, '_', dars$cluster)
ccans <- left_join(ccans, dars[, c('avg_log2FC', 'p_val_adj', 'peak_cluster')], by = 'peak_cluster')


In [262]:
unique(ccans$avg_log2FC)

[1]         NA -1.3331982 -1.5989030 -2.6912772 -1.7683097 -2.1032416
  [7] -1.5588820 -1.8872277 -2.0058444 -1.0488271 -1.6711426 -1.8843048
 [13] -2.2054137 -1.5601910 -1.3657365 -1.7731069 -1.2643428 -1.7248644
 [19] -2.4628556 -2.6518641 -2.8033252 -0.9974196 -2.0553736 -1.2644580
 [25] -1.6629908 -0.8809050 -1.8836337 -1.4971880 -1.6301121 -1.4861038
 [31] -1.6998401 -1.8516163 -1.2656959 -2.3506523 -1.6321805 -1.6375467
 [37] -2.6158507 -1.1956660 -1.6541029 -1.4294995 -1.5619558 -1.8331821
 [43] -1.3131080 -1.5350949 -0.9585827 -1.0316271 -0.8477824 -2.1725408
 [49] -1.9616813 -2.1644223 -1.7260546 -1.5540662 -1.6759232 -1.6349270
 [55] -2.1732345 -1.9295832 -2.1080378 -1.5438564 -2.1535119 -1.5940537
 [61] -1.5848254 -2.1635703 -1.3595509 -2.2057282 -2.2523990 -1.1241981
 [67] -1.8642388 -1.5206854 -1.4296524 -1.0897141 -2.8735624 -1.7314382
 [73] -1.5126983 -1.6382097 -1.8166908 -1.6586224 -1.0595986 -2.2602617
 [79] -1.7645534 -1.6093401 -2.2751033 -1.4488853 -1.7006140 -1.8572473
 [85] -2.0230343 -1.9531222 -2.3438599 -1.2132581 -1.3492113 -1.5472846
 [91] -1.4959727 -1.3395337 -1.5588589 -1.6391234 -1.7504768 -1.1392718
 [97] -1.5476377 -1.1567960 -1.3877836 -2.3351897 -1.4017841 -1.7606065
[103] -1.4057677 -1.9350808 -1.8405706 -4.9516893 -3.7207792 -1.3430715
[109] -2.0863392 -1.8326553 -1.9129051 -3.7604551 -1.6943457 -1.8645482
[115] -1.6680579 -2.9954246 -2.4431838 -1.7483978 -1.6433770 -1.6026805
[121] -1.6548054 -2.6176625 -2.0159956 -1.3002884 -1.6387901 -1.2026139
[127] -1.0147721 -1.4394299 -1.7609546 -0.9761238 -1.4366378 -2.2692731
[133] -2.1873966 -1.6372438 -2.0121604 -1.7726396 -1.6263942 -1.0110543
[139] -2.1832383 -1.8354872 -0.7834143 -1.2959205 -2.4212952 -2.4579975
[145] -1.4263065 -1.1356936 -1.3106058 -1.1561125 -1.2692196  0.7348628
[151]  0.4254192  0.7892973 -1.0091447 -1.1182932  0.5015879 -1.4951721
[157]  0.4527373  0.5951472  0.3742528  0.4015365  0.4695358  0.3071514
[163]  0.9270934  0.3901708  0.3537815 -1.1678519  0.8359465  0.5356101
[169]  0.3824088  0.4763849  0.3545993  0.3181027  0.8285374  1.0355134
[175]  0.3902927 -1.0057592  0.6005564 -1.0291235  0.4864778  0.6909025
[181]  0.3222852  0.5533132 -0.8849200  0.3064114  0.4150418 -1.0581572
[187]  0.5932694  0.4293267  0.3530161  0.3118129  0.5450003  0.5219754
[193]  0.4041244  0.3789623  0.4517162  0.3442997 -1.2552964  0.6350011
[199]  0.4655010 -1.0303656  0.4700318  0.4788645 -1.1001171  0.7583561
[205]  0.8364301  0.3836953  0.3587165  0.4157724  0.5865448 -0.7641453
[211] -1.7027578  0.3936494  0.3838833  0.3220292  0.3429120  0.7606500
[217]  0.4180113 -1.6880807  0.4087191  0.5910913 -1.0016272  0.5491819
[223]  0.5831650  0.3980231 -0.8812528  0.3135898  0.6950531  0.4830623
[229] -1.3694088  0.5394121  0.3631820 -0.7360206  0.4804205  0.4578630
[235] -0.5696247 -1.2477997  0.4644746 -0.7093696  0.5360785  0.6357035
[241]  0.3020018  0.3764702  0.8821571  1.1227548  0.3857858  0.3860330
[247] -1.9361102  0.5123061  0.6372838  0.5611301  0.5288887  0.3877350
[253]  0.5587687  0.5846571  0.7173491 -1.1071486  0.3393789  0.6852667
[259]  0.6583603  0.5104044  0.5325915 -0.8974953  0.8796882  0.3599966
[265] -0.9416126 -1.1366252 -1.3023352  0.3994703  0.5351819  0.5806953
[271]  0.6097061 -1.0803080 -1.1456622  0.3683158  0.3795474  0.4105434
[277]  0.5737988 -1.1122066  0.4366650  0.3997769  0.4649432 -0.8121470
[283]  0.3567645  0.3893103 -1.1815414 -1.0177892  0.3600955 -1.0911357
[289]  0.5181221  0.3784046  0.5920727  0.7472683  0.6244491  0.3129385
[295]  0.5238631  0.5852127  0.3117724 -1.5667702  2.3606448  0.5350095
[301] -1.5580011  0.6142510  0.3449070  0.9532654  0.4818411  0.4240929
[307] -0.8418890  0.3838944  0.4236824  0.4236176  0.6247775  0.4254840
[313]  0.4091076  0.4219319 -1.1143824 -1.3791744  0.5941953  0.4400368
[319]  0.8743248 -1.4611400  0.4071325  0.4423396 -1.3413719  0.8658079
[325] -0.8375969 -1.1959253 -1.0344604  0.4139357 -0.9076728  0.4928551
[331] -1.0133981  0.3556399 -1.4755902  0.

In [ ]:
download.file(url = "https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M10/gencode.vM10.annotation.gtf.gz", destfile = "tmp_dir/mm10_primary_assembly.annotation.gtf", method = "wget")

In [264]:
mm10 <- rtracklayer::import('tmp_dir/mm10_primary_assembly.annotation.gtf')
deg_coords <- mm10[mm10$gene_name %in% degs$gene]
deg_exons <- deg_coords[deg_coords$type == 'exon',]
deg_exons <- split(deg_exons, f = deg_exons$gene_name)
deg_exons <- lapply(deg_exons, reduce)
deg_exons <- deg_exons[sapply(deg_exons, length) > 1]
deg_introns <- lapply(deg_exons, 
                      function(x) {
                        gr = GRanges(seqnames = seqnames(x)[1], 
                                     ranges = IRanges(start=min(start(x)),end=max(end(x))), 
                                     strand = strand(x)[1])
                        db = disjoin(c(x, gr))
                        ints = db[countOverlaps(db, x) == 0]
                        if(as.character(strand(ints)[1]) == "-") {
                          ints$intron_id = c(length(ints):1)
                        } else {
                          ints$intron_id = c(1:length(ints))
                        }
                        ints
                      })
intron1 <- lapply(1:length(deg_introns), 
                  function(x){
                    deg_introns[[x]]$gene_name <- names(deg_introns)[x]
                    deg_introns[[x]] <- deg_introns[[x]][deg_introns[[x]]$intron_id == 1,]
                    return(deg_introns[[x]])
                  })
intron1 <- GRangesList(intron1)
intron1 <- unlist(intron1)
intron1$type <- 'intron'

deg_promoters <- promoters(deg_coords[deg_coords$type == 'gene',])
deg_promoters$type <- 'promoter'
deg_cre <- sort(c(deg_promoters, intron1), ignore.strand = F)
saveRDS(deg_cre, 'annotated/annotate_ccans_deg_promo_intron1_fctx.rds')

In [266]:
deg_cre$coords <- GRangesToString(deg_cre)

ccans <- split(ccans, f = ccans$cluster)
degs <- split(degs, f = degs$cluster)

ccans <- ccans[names(degs)] # there are less deg clusters
nrow(bind_rows(ccans))
# [1] 180407 <- some were removed bc of removing clusters

for (i in 1:length(ccans)){
  cluster <- names(ccans)[i]
  
  ccans_tmp <- ccans[[cluster]]
  degs_tmp <- degs[[cluster]]
  
  message(cluster)
  genes <- degs_tmp$gene
  deg_ranges <- deg_cre[deg_cre$gene_name %in% genes ]
  ccan_ranges <- ccans_tmp$Peak
  ccan_ranges <- StringToGRanges(ccan_ranges)
  ovrlp <- findOverlaps(query = ccan_ranges, subject = deg_ranges)
  
  deg_hits <- subjectHits(ovrlp)
  ccan_hits <- queryHits(ovrlp)
  
  ccans_tmp$deg_overlap <- ''
  ccans_tmp$deg_logfc <- NA
  ccans_tmp$deg_overlap_type <- ''
  ccans_tmp$deg_overlap_coords <- ''
  
  ccans_tmp$deg_overlap[ccan_hits] <- deg_ranges[deg_hits]$gene_name
  ccans_tmp$deg_overlap_type[ccan_hits] <- deg_ranges[deg_hits]$type
  ccans_tmp$deg_overlap_coords[ccan_hits] <- deg_ranges[deg_hits]$coords
  
  rownames(degs_tmp) <- degs_tmp$gene
  ccans_tmp$deg_logfc[ccan_hits] <- degs_tmp[deg_ranges$gene_name[deg_hits], 'avg_log2FC']
  
  ccans[[cluster]] <- ccans_tmp
}

ccans <- bind_rows(ccans)

[1] 834341

Astro

L2/3 IT CTX

L4/5 IT CTX

L6 CT CTX

Micro

Oligo



In [267]:
dar_deg_ccans <- ccans[ccans$deg_overlap != '', 'ccan_cluster']
dar_deg_ccans <- ccans[ccans$ccan_cluster %in% dar_deg_ccans, ]
dar_deg_ccans <- split(dar_deg_ccans, f = dar_deg_ccans$ccan_cluster)

In [271]:
colnames(dar_deg_ccans$`10033_Astro`)

[1] "Peak"               "CCAN"               "cluster"           
 [4] "region"             "peak_cluster"       "ccan_cluster"      
 [7] "avg_log2FC"         "p_val_adj"          "deg_overlap"       
[10] "deg_logfc"          "deg_overlap_type"   "deg_overlap_coords"

In [275]:
dar_deg_ccans <- lapply(dar_deg_ccans, function(x) {
  dar_fc <- x[x$p_val_adj <= 0.05, 'avg_log2FC']
  deg_fc <- x[!is.na(x$deg_logfc), 'deg_logfc']
  
  if (length(dar_fc) == 0 || length(deg_fc) == 0) {
    x$directionality <- NA
    return(x)
  }
  
  direction <- expand.grid(dar_fc, deg_fc)
  cond <- direction[, 1] * direction[, 2] > 0
  
  uni_cond <- all(cond, na.rm = TRUE) # Ignore NA values
  bi_cond <- all(!cond, na.rm = TRUE) # Ignore NA values
  mix_cond <- length(unique(cond)) > 1
  
  directionality <- "undefined"
  if (uni_cond) {
    directionality <- 'unidirectional'
  }
  if (bi_cond) {
    directionality <- 'bidirectional'
  }
  if (mix_cond) {
    directionality <- 'mixed'
  }
  
  x$directionality <- directionality
  return(x)
})

dar_deg_ccans <- bind_rows(dar_deg_ccans)

In [282]:
saveRDS(ccans, 'annotated/annotated_ccans.rds')
saveRDS(dar_deg_ccans, 'annotated/annotate_ccans_annotated.ccans.rds')

In [303]:
filtered_df <- dar_deg_ccans[!is.na(dar_deg_ccans$avg_log2FC) & !is.na(dar_deg_ccans$deg_logfc), ]

In [304]:
filtered_df

,Peak,CCAN,cluster,region,peak_cluster,ccan_cluster,avg_log2FC,p_val_adj,deg_overlap,deg_logfc,deg_overlap_type,deg_overlap_coords,directionality
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
9181,chr8-10928169-10929025,10945,Astro,fctx,chr8-10928169-10929025_Astro,10945_Astro,-1.4366378,9.949009e-04,Fth1,-0.3901525,intron,chr8-9982703-86915156,mixed
9328,chr8-11913028-11914126,10945,Astro,fctx,chr8-11913028-11914126_Astro,10945_Astro,-2.2692731,1.096517e-08,Fth1,-0.3901525,intron,chr8-9982703-86915156,mixed
14305,chr8-24437640-24439452,11182,Astro,fctx,chr8-24437640-24439452_Astro,11182_Astro,-2.0121604,4.931188e-04,Fth1,-0.3901525,intron,chr8-9982703-86915156,mixed
14503,chr8-24987225-24988003,11185,Oligo,fctx,chr8-24987225-24988003_Oligo,11185_Oligo,-2.2730465,1.018542e-05,Fth1,-0.3336068,intron,chr8-9982703-86915156,mixed
14672,chr8-26748203-26749113,11191,Astro,fctx,chr8-26748203-26749113_Astro,11191_Astro,-1.7726396,1.242489e-02,Fth1,-0.3901525,intron,chr8-9982703-86915156,mixed
14747,chr8-27606681-27607740,11191,Astro,fctx,chr8-27606681-27607740_Astro,11191_Astro,-1.6263942,6.029978e-03,Fth1,-0.3901525,intron,chr8-9982703-86915156,mixed
16385,chr8-57455579-57457599,11332,Astro,fctx,chr8-57455579-57457599_Astro,11332_Astro,-1.0110543,3.428860e-02,Scrg1,-0.3112355,intron,chr8-57456128-57474305,mixed
16597,chr8-64880339-64881734,11357,Astro,fctx,chr8-64880339-64881734_Astro,11357_Astro,-2.1832383,7.537022e-07,Fth1,-0.3901525,intron,chr8-9982703-86915156,mixed
17800,chr8-70583462-70584482,11427,Oligo,fctx,chr8-70583462-70584482_Oligo,11427_Oligo,-1.4309811,2.052583e-05,Fth1,-0.3336068,intron,chr8-9982703-86915156,mixed


In [305]:
library(data.table)
findCCRES <- function(uni_or_mixed, fc_cut){
  
  regulatory_ccans <- readRDS('annotated/annotate_ccans_annotated.ccans.rds') %>% as.data.table() # only includes ccans with dap/deg overlaps
  regulatory_ccans <- regulatory_ccans[directionality == uni_or_mixed, ]
  
  regulatory_ccans$peak_ccan <- regulatory_ccans[, paste0(Peak, '_', ccan_cluster)]
  ccan_daps <- regulatory_ccans[p_val_adj <= 0.05, ] # all daps in any ccan

  pro_daps <- regulatory_ccans[p_val_adj <= 0.05, ] 
  pro_daps <- pro_daps[deg_overlap != '', ]
  pro_daps <- pro_daps[abs(deg_logfc) >= fc_cut,] 
  
  # overlapped deg and dap have same sign logFC
  pro_daps <- pro_daps[deg_logfc*avg_log2FC > 0, ]
  
  cluster <- unique(ccan_daps$cluster)
  conns <- lapply(cluster, 
                  function(x){
										gsub('/', '_', x)	
                    message(x)
                    # both peaks are daps
                    tmp <- ccan_daps[cluster == x, ]
                    conns_x <- paste0('CCANs/', x, '_fctx_CCANs.rds')
                    conns_x <- readRDS(conns_x) %>% as.data.table()
                    conns_x$Peak2 <- as.character(conns_x$Peak2)
                    conns_x <- conns_x[Peak1 %in% tmp$Peak, ]
                    conns_x <- conns_x[Peak2 %in% tmp$Peak, ]
                    
                    tmp <- tmp[, .(Peak, peak_ccan, ccan_cluster)]
                    conns_x <- left_join(x = conns_x, y = tmp, by = c('Peak1' = 'Peak'))
                    conns_x <- left_join(x = conns_x, y = tmp, by = c('Peak2' = 'Peak'))
                    colnames(conns_x) <- gsub('.x', '1', colnames(conns_x))
                    colnames(conns_x) <- gsub('.y', '2', colnames(conns_x))
                    conns_x <- conns_x[ccan_cluster1 == ccan_cluster2, ]
                    return(conns_x)
                  })
  conns <- conns[sapply(conns, nrow) > 0] %>% bind_rows()

  # one of the peaks overlaps a deg
  conns <- conns[peak_ccan1 %in% pro_daps$peak_ccan | peak_ccan2 %in% pro_daps$peak_ccan, ]
  
  # merge with rest of data
  conns <- left_join(x = conns, y = regulatory_ccans[, .(peak_ccan, avg_log2FC)], by = c('peak_ccan1' = 'peak_ccan'))
  conns <- left_join(x = conns, y = regulatory_ccans[, .(peak_ccan, avg_log2FC)], by = c('peak_ccan2' = 'peak_ccan'))

  # both daps go same direction
  conns <- conns[avg_log2FC.x*avg_log2FC.y > 0, ]
  
  cherrypicker <- unique(c(conns$peak_ccan1, conns$peak_ccan2))
  ccres <- regulatory_ccans[peak_ccan %in% cherrypicker, ]
  
  # n degs 
  n_degs <- ccres[deg_overlap != '', unique(deg_overlap), by = cluster][, .N, by = cluster]
  # names
  deg_names <- ccres[deg_overlap != '', paste(unique(deg_overlap), collapse = ', '), by = cluster]
  # n ccans
  n_ccans <- ccres[deg_overlap != '', unique(ccan_cluster), by = cluster][, .N, by = cluster]
  
  ccres <- full_join(x = n_ccans, y = n_degs, by = 'cluster')
  ccres <- full_join(x = ccres, y = deg_names, by = 'cluster')
  colnames(ccres) <- c('cluster', 'n_ccans', 'n_degs', 'deg_names')
  ccres$degs <- paste0(ccres$n_degs, '\n', ccres$deg_names)
  ccres <- ccres[, .(cluster, n_ccans, degs)]
  return(ccres)
}
fc <- 0.3

uni <- findCCRES(uni_or_mixed = 'unidirectional', fc_cut = fc)
mix <- findCCRES(uni_or_mixed = 'mixed', fc_cut = fc)

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function '%in%': object 'peak_ccan1' not found
